# Logistic Regression & Calibration — Student Lab

We focus on *probabilities*, not just accuracy.

In [1]:
# Caliberation:  how accurate our predicted probabilities are as compared to actual outcomes
# Model is well caliberated when for all the prediction withg probability p, p fraction of them belong to positive class
# Imbalace data refers to the fact when i have classes which are not equally represented in the dataset
# Ex: email is spam or not spam, imbalance data look like this : 95% not spam, 5% spam
# Imbalance data cause many problems for ML models because they tend to be biased towards the majority class
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

rng = np.random.default_rng(0)

## Section 0 — Synthetic imbalanced data
We simulate logits and labels with imbalance and miscalibration.

In [ ]:
# Base_rate = 0.05 means 5% of the data belongs to positive class
# logit_scale controls the spread of the probabilities
# miscalibration > 1 means model is overconfident, < 1 means underconfident
def make_probs(n=5000, base_rate=0.05, logit_scale=1.0, miscalibration=1.0):
    # Generate true probabilities via latent logits
    z = logit_scale * rng.standard_normal(n)
    # shift to get desired base rate approximately
    z = z + np.log(base_rate/(1-base_rate))
    p_true = 1/(1+np.exp(-z)) # true probabilities
    y = (rng.random(n) < p_true).astype(int) # generates random numbers between 0 and 1,If it’s less than p_true, call it 1 else 0
    z_model = miscalibration * z
    p_model = 1/(1+np.exp(-z_model))
    return y, p_model, p_true

y, p_model, p_true = make_probs(miscalibration=2.0) 
print('base_rate', y.mean())
check('in_0_1', np.all((p_model>=0) & (p_model<=1)))

base_rate 0.0712
OK: in_0_1


## Section 1 — Metrics

### Task 1.1: Confusion matrix metrics at a threshold
Implement precision, recall, F1 at threshold t.

# HINT:
- y_hat = (p>=t)
- TP/FP/FN

**Checkpoint:** Why is accuracy misleading under imbalance?

In [ ]:
# Confusion matrix means at a given threshold t, how many true positives, true negatives, false positives, false negatives we have
# We use it to evaluate classification models performance


def metrics_at_threshold(y, p, t):
    # TODO
    y = y.astype(int)
    yhat = (p >= t).astype(int)
    tp = int(np.sum((y == 1) & (yhat == 1)) )# true positives
    fp = int(np.sum((y == 0) & (yhat == 1)) )# false positives
    fn = int(np.sum((y == 1) & (yhat == 0)) )# false negatives
    prec = tp / (tp + fp + 1e-12 )  # precision = of all emails predicted as spam, how many were actually spam. High precison = few false alarms
    rec = tp / (tp + fn + 1e-12 )   # recall = of all actual spam emails, how many were predicted as spam, High recall = miss very little spam
    f1 = 2 * (prec * rec) / (prec + rec + 1e-12)  # F1 score
    return {'tp': tp, 'fp': fp, 'fn': fn, 'precision': prec, 'recall': rec, 'f1': f1} 
   
m = metrics_at_threshold(y, p_model, t=0.5) 
print(m)

{'tp': 2, 'fp': 2, 'fn': 354, 'precision': 0.499999999999875, 'recall': 0.005617977528089872, 'f1': 0.011111111111089075}


### Task 1.2: PR curve area (approx)
Compute a simple PR-AUC approximation by sorting thresholds.

# HINT:
- sort by p desc
- compute precision/recall at each cut

**Interview Angle:** when is PR-AUC preferable to ROC-AUC?

In [4]:
# Reacall on x axis, Precision on y axis
# PR curve area : single number summary of the precision-recall curve
# it also measures the ability of the model to rank positive examples higher than negative examples
# ROC curve area : it's used to evaluate the performance of binary classification models
def pr_curve(y, p):
    # TODO: return arrays (recall, precision)
    order = np.argsort(-p)  # sort by predicted probabilities in descending order
    y_sorted = y[order]
    tp = np.cumsum(y_sorted == 1)  # true positives
    fp = np.cumsum(y_sorted == 0) # false positives
    prec = tp / (tp + fp + 1e-12) # precision means how many selected items are relevant
    rec = tp / (tp[-1] + 1e-12) # recall means how many relevant items are selected
    return rec, prec

def auc_trapz(x, y):
    # TODO
    return float(np.trapz(y, x))  # area under curve using trapezoidal rule

rec, prec = pr_curve(y, p_model)
pr_auc = auc_trapz(rec, prec)
print('pr_auc', pr_auc)
check('finite', np.isfinite(pr_auc))

pr_auc 0.20170158178004188
OK: finite


## Section 2 — Calibration

### Task 2.1: Reliability curve + ECE

Bin probabilities and compute:
- avg predicted prob per bin
- empirical accuracy per bin
- ECE = sum (bin_weight * |acc - conf|)

# HINT:
- np.digitize

**FAANG gotcha:** model can have good ranking but bad calibration.

In [8]:
# Caliberation : means how accurate our predicted probabilities are as compared to actual outcomes
# the models predicted probabilities should match the actual outcomes
# suppose model predicts 0.7(70%) probability , does that 70% really happen? 
# x-axis : predicted probability, y-axis : observed frequency
# reliability diagram : plot of observed frequency vs predicted probability
def reliability_bins(y, p, n_bins=10):
    # TODO: return (bin_acc, bin_conf, bin_frac)
    y = y.astype(int)
    edges = np.linspace(0, 1, n_bins + 1)
    b = np.digitize(p, edges[1:-1], right=False)  # bin indices
    bin_acc = np.zeros(n_bins)  # accuracy in each bin
    bin_conf = np.zeros(n_bins)  # average predicted prob in each bin
    bin_frac = np.zeros(n_bins)  # fraction of samples in each bin
    for i in range(n_bins):
        mask = (b == i)
        if mask.any():
            bin_acc[i] = y[mask].mean()
            bin_conf[i] = p[mask].mean()
            bin_frac[i] = mask.mean()

    return bin_acc, bin_conf, bin_frac


def ece(bin_acc, bin_conf, bin_frac):
    # TODO
    return float(np.sum(bin_frac * np.abs(bin_acc - bin_conf)))  # expected calibration error

bin_acc, bin_conf, bin_frac = reliability_bins(y, p_model, n_bins=10)
ECE = ece(bin_acc, bin_conf, bin_frac)
print('ECE', ECE)
check('ece_finite', np.isfinite(ECE))

ECE 0.056267450617459955
OK: ece_finite


### Task 2.2: Temperature scaling

We assume p_model came from logits z_model. Approximate logits via logit(p).
Then find temperature T that minimizes NLL on validation split: sigmoid(z/T).

# HINT:
- logit(p)=log(p/(1-p))
- grid search T over [0.5..5]

**Checkpoint:** Why does scaling logits preserve ranking?

In [13]:
# its a simple technique to improve calibration of a model by scaling the logits
# temperature scaling : divide logits by a temperature parameter T
# T > 1 means we are making the model less confident
# T < 1 means we are making the model more confident
def logit(p, eps=1e-12):
    p = np.clip(p, eps, 1-eps)
    return np.log(p/(1-p))

def nll(y, p, eps=1e-12):
    p = np.clip(p, eps, 1-eps)
    return float(-np.mean(y*np.log(p) + (1-y)*np.log(1-p)))

# TODO: split into val and fit T
idx = rng.permutation(len(y))
val = idx[: len(y)//2]
test = idx[len(y)//2:]

z = logit(p_model)

ts = np.linspace(0.5, 5.0, 50)
best_T = None
best_loss = float('inf')
for T in ts:
    pT = 1/(1+np.exp(-(z[val]/T)))
    loss = nll(y[val], pT)
    if loss < best_loss:
        best_loss = loss
        best_T = T
pcal = 1/(1+np.exp(-z[test]/best_T))
ECE_before = ece(*reliability_bins(y[test], p_model[test], 10))
ECE_after = ece(*reliability_bins(y[test], pcal, 10))


print('best_T', best_T)
# apply temperature on test
p_cal = 1/(1+np.exp(-(z[test]/best_T)))
ECE_before = ece(*reliability_bins(y[test], p_model[test], 10))
ECE_after = ece(*reliability_bins(y[test], p_cal, 10))
print('ECE_before', ECE_before, 'ECE_after', ECE_after)

best_T 2.0612244897959187
ECE_before 0.051093775041383536 ECE_after 0.014975308183525216


## Section 3 — Thresholding with costs

### Task 3.1: Pick threshold minimizing cost
Cost = c_fp*FP + c_fn*FN

# TODO: sweep thresholds and pick best.

**Interview Angle:** map model probabilities to business decisions.

In [14]:
def best_threshold_cost(y, p, c_fp=1.0, c_fn=10.0):
    # TODO
    ts = np.linspace(0, 1, 501)
    best_t = 0.5
    best_cost = float('inf')
    for t in ts:
        y_hat = (p >= t).astype(int)
        fp = np.sum((y == 0) & (y_hat == 1))
        fn = np.sum((y == 1) & (y_hat == 0))
        cost = c_fp * fp + c_fn * fn
        if cost < best_cost:
            best_cost = cost
            best_t = t
    return best_t, best_cost
  

t_star, cost_star = best_threshold_cost(y, p_model, c_fp=1.0, c_fn=10.0)
print('t*', t_star, 'cost', cost_star)

t* 0.006 cost 2626.0


---
## Submission Checklist
- All TODOs completed
- ECE computed
- Temperature scaling applied
- Threshold recommendation written
